In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weather-dataset2/weather_dataset.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a355a854150f52e68cda33076b70d2dcb6c156ddd32f37a1f1df6d5b739a2564
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel
import os

# Initialize Spark Session
spark = SparkSession.builder.appName('weather_prediction').getOrCreate()

# Load the dataset
file_path = '/kaggle/input/weather-dataset/weather_dataset.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 13:54:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Split the data into training and test sets
# Chia bộ dữ liệu thành hai phần: dữ liệu huấn luyện và dữ liệu kiểm thử.
train_data, test_data = df.randomSplit([0.8, 0.2])

In [5]:
# Selecting relevant columns and dropping rows with missing values
# Chọn ra các cột dữ liệu và loại bỏ những hàng có giá trị thiếu.
selected_columns = ['sunrise','sunset','moonrise','moonset','moon_phase','moon_illumination','time','tempC','tempF','windspeedMiles','windspeedKmph','winddirDegree','winddir16Point','precipInches','humidity','visibility','visibilityMiles','pressure','pressureInches','cloudcover','HeatIndexC','HeatIndexF','DewPointC','DewPointC','WindChillC','WindChillF','WindGustMiles','WindGustKmph','FeelsLikeC','FeelsLikeF','uvIndex', 'predict']
df = df.select(selected_columns).na.drop()

In [6]:
# Indexing the label column
indexer = StringIndexer(inputCol='predict', outputCol='label')
predictStringIndexer = indexer.fit(df)
df = predictStringIndexer.transform(df)

24/01/21 13:54:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
indexer = StringIndexer(inputCol='sunrise', outputCol='sunriseIndex')
sunriseIndexIndexer = indexer.fit(df)
df = sunriseIndexIndexer.transform(df)

indexer = StringIndexer(inputCol='sunset', outputCol='sunsetIndex')
sunsetStringIndexer = indexer.fit(df)
df = sunsetStringIndexer.transform(df)

indexer = StringIndexer(inputCol='moonrise', outputCol='moonriseIndex')
moonriseStringIndexer = indexer.fit(df)
df = moonriseStringIndexer.transform(df)

indexer = StringIndexer(inputCol='moonset', outputCol='moonsetIndex')
moonsetStringIndexer = indexer.fit(df)
df = moonsetStringIndexer.transform(df)

indexer = StringIndexer(inputCol='moon_phase', outputCol='moon_phaseIndex')
moon_phaseStringIndexer = indexer.fit(df)
df = moon_phaseStringIndexer.transform(df)

indexer = StringIndexer(inputCol='winddir16Point', outputCol='WIndex')
winddir16PointStringIndexer = indexer.fit(df)
df = winddir16PointStringIndexer.transform(df)

In [8]:
df.select('sunriseIndex', 'sunsetIndex','moonriseIndex','moonsetIndex','moon_phaseIndex','WIndex','label' ).show()

+------------+-----------+-------------+------------+---------------+------+-----+
|sunriseIndex|sunsetIndex|moonriseIndex|moonsetIndex|moon_phaseIndex|WIndex|label|
+------------+-----------+-------------+------------+---------------+------+-----+
|         7.0|        0.0|       1112.0|       328.0|            5.0|   1.0|  1.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   1.0|  0.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   1.0|  0.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   1.0|  0.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   1.0|  0.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   1.0|  1.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   0.0|  1.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   0.0|  1.0|
|         7.0|        0.0|       1112.0|       328.0|            5.0|   0.0|  0.0|
|   

In [10]:
# Assembling the features
assembler = VectorAssembler(inputCols=['sunriseIndex','sunsetIndex','moonriseIndex','moonsetIndex','moon_phaseIndex','moon_illumination','time','tempC','tempF','windspeedMiles','windspeedKmph','winddirDegree','WIndex','precipInches','humidity','visibility','visibilityMiles','pressure','pressureInches','cloudcover','HeatIndexC','HeatIndexF','DewPointC','DewPointC','WindChillC','WindChillF','WindGustMiles','WindGustKmph','FeelsLikeC','FeelsLikeF','uvIndex'], outputCol='features')
df = assembler.transform(df)

In [12]:
# Initialize classifiers
# Khởi tạo hai mô hình phân loại: Random Forest và Logistic Regression.
rf = RandomForestClassifier(labelCol='label', featuresCol='features', maxBins=1500)
lr = LogisticRegression(labelCol='label', featuresCol='features')

In [13]:
# Initialize pipeline
# Xây dựng pipeline cho mỗi mô hình để huấn luyện trên dữ liệu.
pipeline_rf = Pipeline(stages=[predictStringIndexer, sunriseIndexIndexer, sunsetStringIndexer, moonriseStringIndexer, moonsetStringIndexer, moon_phaseStringIndexer, winddir16PointStringIndexer, assembler, rf])
pipeline_lr = Pipeline(stages=[predictStringIndexer, sunriseIndexIndexer, sunsetStringIndexer, moonriseStringIndexer, moonsetStringIndexer, moon_phaseStringIndexer, winddir16PointStringIndexer, assembler, lr])

In [ ]:
# Fit the models
# Huấn luyện 2 mô hình
model_rf = pipeline_rf.fit(train_data)
model_lr = pipeline_lr.fit(train_data)

In [15]:
# Save the Random Forest model
model_rf.save("/kaggle/working/random_forest_model")

In [16]:
# Save the Logistic Regression model
model_lr.save("/kaggle/working/logistic_regression_model")

In [17]:
# Make predictions
# Dự đoán trên một tập test
predictions_rf = model_rf.transform(test_data)
predictions_lr = model_lr.transform(test_data)

In [18]:
predictions_rf['predict','prediction', 'rawPrediction'].show()

+-------+----------+--------------------+
|predict|prediction|       rawPrediction|
+-------+----------+--------------------+
|no rain|       0.0|[18.0400077891604...|
|   rain|       1.0|[9.41344602625256...|
|no rain|       0.0|[17.8218118808155...|
|   rain|       1.0|[2.48780644703998...|
|   rain|       1.0|[2.02557028543309...|
|no rain|       0.0|[12.1257311992066...|
|no rain|       0.0|[12.1257311992066...|
|no rain|       0.0|[12.1257311992066...|
|no rain|       0.0|[12.1257311992066...|
|   rain|       0.0|[10.7378993756633...|
|   rain|       1.0|[9.33061704085989...|
|   rain|       1.0|[2.57651140750210...|
|no rain|       0.0|[17.2218002220025...|
|   rain|       1.0|[7.97650992056048...|
|no rain|       0.0|[16.8413431592400...|
|no rain|       0.0|[17.6684719912031...|
|no rain|       0.0|[10.3587221954229...|
|no rain|       0.0|[10.4477746599654...|
|no rain|       0.0|[10.3587221954229...|
|no rain|       0.0|[17.6002231146820...|
+-------+----------+--------------

In [20]:
df.show()

+--------+--------+--------+--------+---------------+-----------------+----+-----+-----+--------------+-------------+-------------+--------------+------------+--------+----------+---------------+--------+--------------+----------+----------+----------+---------+---------+----------+----------+-------------+------------+----------+----------+-------+-------+-----+------------+-----------+-------------+------------+---------------+------+--------------------+
| sunrise|  sunset|moonrise| moonset|     moon_phase|moon_illumination|time|tempC|tempF|windspeedMiles|windspeedKmph|winddirDegree|winddir16Point|precipInches|humidity|visibility|visibilityMiles|pressure|pressureInches|cloudcover|HeatIndexC|HeatIndexF|DewPointC|DewPointC|WindChillC|WindChillF|WindGustMiles|WindGustKmph|FeelsLikeC|FeelsLikeF|uvIndex|predict|label|sunriseIndex|sunsetIndex|moonriseIndex|moonsetIndex|moon_phaseIndex|WIndex|            features|
+--------+--------+--------+--------+---------------+-----------------+----+

In [21]:
# Initialize evaluator
# Khởi tạo bộ đánh giá
# Sử dụng để đánh giá độ chính xác của mô hình phân loại đa lớp bằng cách so sánh các nhãn dự đoán với các nhãn thực tế trong tập dữ liệu
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [22]:
# Evaluate the models
# Đánh giá độ chính xác của hai mô hình
accuracy_rf = evaluator.evaluate(predictions_rf)
accuracy_lr = evaluator.evaluate(predictions_lr)

In [23]:
# Print the results
print('Random Forest Accuracy:', accuracy_rf)
print('Logistic Regression Accuracy:', accuracy_lr)

Random Forest Accuracy: 0.8365167916651273
Logistic Regression Accuracy: 0.8139801233974951


In [24]:
# Calculate precision, recall, f1-score
# Tính toán các chỉ số precision, recall, và f1-score cho hai mô hình
evaluator.setMetricName('weightedPrecision')
precision_rf = evaluator.evaluate(predictions_rf)
precision_lr = evaluator.evaluate(predictions_lr)

evaluator.setMetricName('weightedRecall')
recall_rf = evaluator.evaluate(predictions_rf)
recall_lr = evaluator.evaluate(predictions_lr)

evaluator.setMetricName('f1')
f1_score_rf = evaluator.evaluate(predictions_rf)
f1_score_lr = evaluator.evaluate(predictions_lr)

In [25]:
# Print the evaluation metrics
print('Random Forest Precision:', precision_rf)
print('Random Forest Recall:', recall_rf)
print('Random Forest F1-Score:', f1_score_rf)
print('Logistic Regression Precision:', precision_lr)
print('Logistic Regression Recall:', recall_lr)
print('Logistic Regression F1-Score:', f1_score_lr)

Random Forest Precision: 0.8339644214904203
Random Forest Recall: 0.8365167916651273
Random Forest F1-Score: 0.826309268621003
Logistic Regression Precision: 0.8080739108107163
Logistic Regression Recall: 0.8139801233974951
Logistic Regression F1-Score: 0.8093841060522646


In [26]:
# Stop the Spark session
spark.stop()